# Decoder object

Nilearn has since release 0.7.0 a new decoder object:

- `nilearn.decoding.Decoder` for classification and 
- `nilearn.decoding.DecoderRegressor` for regression

The decoder object implements a model selection scheme that averages the best models within a cross validation loop.

## Outline

- <a href="#decoding">I. What is decoding?</a>
- <a href="#classif">II. Decoder for classification</a>
    - <a href="#classif-example">Example</a>
        - <a href="#classif-example-data">Get the data</a>
        - <a href="#classif-example-decoding">Decoding pipeline</a>
        - <a href="#classif-example-cross">How to perform cross validation?</a>
        - <a href="#classif-example-leave">Leave one group out cross validation</a>
        - <a href="#classif-example-chance">What is the chance level accuracy?</a>
    - <a href="#classif-compare">Why is the decoder useful?</a>
- III. Decoder for Regression

<span id="decoding"></span>

## I - What is decoding?

Decoding can be seen as the task of predicting from fMRI activity. A good example is the Haxby experiment in which subjects were presented visual stimuli from different categories. Decoding in this context means predicting which category the subject is seeing from the fMRI activity recorded in regions of the ventral visual system. Significant prediction shows that the signal in the region contains information on the corresponding category.

**TODO: Add a picture here**

If you are not familiar with decoding, we recommand that you read the *introduction to decoding* tutorial on nilearn's website [here](http://nilearn.github.io/decoding/decoding_intro.html).

<span id="classif"></span>

## II - Decoder for classification

First of all, make sure you have nilearn >= 0.7.0 installed:

In [1]:
import nilearn
print(nilearn.__version__)

0.7.2.dev


The documentation for the classification decoder is available on the website [here](http://nilearn.github.io/modules/generated/nilearn.decoding.Decoder.html#nilearn.decoding.Decoder), or through the Jupyter magic command:

In [2]:
from nilearn.decoding import Decoder
?Decoder

Init signature:
Decoder(
    estimator='svc',
    mask=None,
    cv=10,
    param_grid=None,
    screening_percentile=20,
    scoring='roc_auc',
    smoothing_fwhm=None,
    standardize=True,
    target_affine=None,
    target_shape=None,
    mask_strategy='background',
    low_pass=None,
    high_pass=None,
    t_r=None,
    memory=None,
    memory_level=0,
    n_jobs=1,
    verbose=0,
)
Docstring:     
A wrapper for popular classification strategies in neuroimaging.

The `Decoder` object supports classification methods.
It implements a model selection scheme that averages the best models
within a cross validation loop. The resulting average model is the
one used as a classifier. This object also leverages the`NiftiMaskers` to
provide a direct interface with the Nifti files on disk.

Parameters
-----------
estimator: str, optional
    The estimator to choose among: 'svc', 'svc_l2', 'svc_l1', 'logistic',
    'logistic_l1', 'logistic_l2' and 'ridge_classifier'. Note that
    'svc' and '

<span id="classif-example"></span>

### Example

<span id="classif-example-data"></span>

#### Get the data

As an example, we will use the Haxby dataset:

In [3]:
from nilearn.datasets import fetch_haxby

haxby_dataset = fetch_haxby()
haxby_dataset.keys()

/home/nicolas/GitRepos/nilearn-fork/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


dict_keys(['anat', 'func', 'session_target', 'mask_vt', 'mask_face', 'mask_house', 'mask_face_little', 'mask_house_little', 'mask', 'description'])

In [4]:
from nilearn.image import load_img
load_img(haxby_dataset.func[0]).shape

(40, 64, 64, 1452)

As we can see, the data is quite large and contains many conditions:

In [5]:
# Load behavioral information
import pandas as pd

behavioral = pd.read_csv(haxby_dataset.session_target[0], delimiter=' ')
print(behavioral)

     labels  chunks
0      rest       0
1      rest       0
2      rest       0
3      rest       0
4      rest       0
...     ...     ...
1447   rest      11
1448   rest      11
1449   rest      11
1450   rest      11
1451   rest      11

[1452 rows x 2 columns]


In [6]:
# What conditions do we have in this dataset?
import numpy as np
conditions = behavioral['labels']
np.unique(conditions)

array(['bottle', 'cat', 'chair', 'face', 'house', 'rest', 'scissors',
       'scrambledpix', 'shoe'], dtype=object)

In order to simplify this example, we will focus only on `cats` and `faces` signals. We need to drop all data unrelated to these signals. One way to do that is to create a condition mask:

In [7]:
condition_mask = conditions.isin(['face', 'cat'])

We can then use this condition mask to restrict our analysis to cats and faces:

In [8]:
from nilearn.image import index_img

cats_and_faces_data = index_img(haxby_dataset.func[0], 
                                condition_mask)
load_img(cats_and_faces_data).shape

(40, 64, 64, 216)

In [9]:
conditions = conditions[condition_mask].values
conditions.shape

(216,)

We have now a much smaller dataset to work on.

<span id="classif-example-decoding"></span>

#### Decoding basic pipeline

We now have everything we need to do some decoding.

The Decoder object follows the usual `fit`, `transform`, `predict` pattern from scikit-learn such that you usually follow the following pattern:

- Instanciate the Decoder with chosen parameters (estimator, mask...)
- Fit the decoder on training set
- Predict on testing set

In [10]:
decoder = Decoder(estimator='svc',               # Use Support Vector Classifier 
                  mask=haxby_dataset.mask_vt[0], # Provide a mask of the Ventral Temporal cortex
                  standardize=True)              # Standardize time series

Lets pretend we have a training set and a test set by leaving out the last 30 samples and predicting on these samples:

In [11]:
train = index_img(cats_and_faces_data, slice(0, -30))    # train set is all but last 30 samples
test  = index_img(cats_and_faces_data, slice(-30, None)) # test set is only the last 30 samples
conditions_train = conditions[:-30]
conditions_test = conditions[-30:]

# We fit our decoder on the train set
decoder.fit(train, conditions_train)

# And predict on the test set
prediction = decoder.predict(test)

In order to quantify how well we are doing, we can compute the prediction accuracy on the test set, which is the proportion of correctly classified samples:

In [12]:
print("Prediction Accuracy: {:.3f}".format(
    (prediction == conditions_test).sum() / float(len(conditions_test))))

Prediction Accuracy: 0.767


<span id="classif-example-cross"></span>

#### Cross Validation

The decoder also implements a cross-validation loop by default and returns an array of shape (cross-validation parameters, n_folds). We can use accuracy score to measure its performance by defining accuracy as the scoring parameter.

In [13]:
n_folds = 5
decoder = Decoder(estimator='svc', 
                  mask=haxby_dataset.mask_vt[0],
                  standardize=True,
                  cv=n_folds,
                  scoring='accuracy')
decoder.fit(cats_and_faces_data,
            conditions)

We can then check the scores and best performing params per fold:

In [14]:
decoder.cv_scores_

{'cat': [0.9772727272727273,
  0.7906976744186046,
  0.7441860465116279,
  0.8372093023255814,
  0.813953488372093],
 'face': [0.9772727272727273,
  0.7906976744186046,
  0.7441860465116279,
  0.8372093023255814,
  0.813953488372093]}

In [15]:
decoder.cv_params_

{'cat': {'C': [100.0, 100.0, 100.0, 100.0, 100.0]},
 'face': {'C': [100.0, 100.0, 100.0, 100.0, 100.0]}}

<span id="classif-example-leave"></span>

#### Leave one group out cross validation

The best way to do cross-validation is to respect the structure of the experiment, for instance by leaving out full sessions of acquisition.

The number of the session is stored in the CSV file giving the behavioral data. We have to apply our session mask, to select only cats and faces.

In [16]:
session_label = behavioral['chunks'][condition_mask]

The fMRI data is acquired by sessions, and the noise is autocorrelated in a given session. Hence, it is better to predict across sessions when doing cross-validation. To leave a session out, pass the cross-validator object to the cv parameter of decoder.

In [17]:
from sklearn.model_selection import LeaveOneGroupOut
cv = LeaveOneGroupOut()

decoder = Decoder(estimator='svc', 
                  mask=haxby_dataset.mask_vt[0], 
                  standardize=True,
                  cv=cv)
decoder.fit(cats_and_faces_data, 
            conditions, 
            groups=session_label)
print(decoder.cv_scores_)

{'cat': [1.0, 1.0, 1.0, 1.0, 0.9629629629629629, 0.8518518518518519, 0.9753086419753086, 0.40740740740740744, 0.9876543209876543, 1.0, 0.9259259259259259, 0.8765432098765432], 'face': [1.0, 1.0, 1.0, 1.0, 0.9629629629629629, 0.8518518518518519, 0.9753086419753086, 0.40740740740740744, 0.9876543209876543, 1.0, 0.9259259259259259, 0.8765432098765432]}


<span id="classif-example-chance"></span>

#### Chance level accuracy

Does the model above perform better than chance? 

To answer this question, we measure a score at random using simple strategies that are implemented in the `Decoder` object. This is useful to inspect the decoding performance by comparing to a score at chance.

Let’s define a object with Dummy estimator replacing ‘svc’ for classification setting. This object initializes estimator with default dummy strategy.

In [18]:
dummy_decoder = Decoder(estimator='dummy_classifier', 
                        mask=haxby_dataset.mask_vt[0],
                        cv=cv)
dummy_decoder.fit(cats_and_faces_data, 
                  conditions, 
                  groups=session_label)
# Now, we can compare these scores by simply taking a mean over folds
print(dummy_decoder.cv_scores_)

{'cat': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], 'face': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]}


<span id="classif-compare"></span>

### Why is the decoder object useful?

In this section we will compare the new decoding pipeline resulting from the addition of the decoder object to Nilearn with the previous way of decoding.

We keep the same example as before, but we decode as if the decoder didn't exist...

In [19]:
# First, we need to build X by masking the data
from nilearn.input_data import NiftiMasker
masker = NiftiMasker(haxby_dataset.mask_vt[0])
X = masker.fit_transform(cats_and_faces_data)

In [20]:
# Then, we preprocess our labels to build y
from sklearn.preprocessing import LabelBinarizer
target_encoder = LabelBinarizer(pos_label=1, 
                                neg_label=-1)
y = target_encoder.fit_transform(conditions)
assert X.shape[0] == y.shape[0] # Make sure we have as many samples as we have labels...

In [21]:
# Define the cross validation strategy
cv_object = LeaveOneGroupOut()
cv_folds  = list(cv_object.split(X, y, 
                                 groups=session_label))

In [22]:
# Define and parameterize our estimator (use SVC here, just like above...)
from sklearn.svm import LinearSVC
estimator = LinearSVC(penalty='l2', # these are the default parameters
                      max_iter=1e4) # used in nilearn decoding

In [23]:
# Define a scoring strategy
from sklearn.metrics import get_scorer
scorer = get_scorer("accuracy")

In [24]:
from sklearn import clone
scores = []; coefs  = []; intercept = []
for train,test in cv_folds:
    X_train, y_train = X[train], y[:,0][train]
    X_test, y_test   = X[test], y[:,0][test]
    estimator_ = clone(estimator)
    estimator_.fit(X_train, y_train)
    score = scorer(estimator_, X_test, y_test)
    scores.append(score)
    coefs.append(estimator_.coef_)
    intercept.append(estimator_.intercept_)

/home/nicolas/anaconda3/envs/nilearn/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/nicolas/anaconda3/envs/nilearn/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/nicolas/anaconda3/envs/nilearn/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/nicolas/anaconda3/envs/nilearn/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/nicolas/anaconda3/envs/nilearn/lib/python3.8/site-packages

In [25]:
aggregated_coeff = np.mean(coefs, axis=0)
aggregated_intercept = np.mean(intercept, axis=0)

In [26]:
from sklearn.utils.extmath import safe_sparse_dot
scores = safe_sparse_dot(X, aggregated_coeff.T, dense_output=True) + aggregated_intercept

In [27]:
indices = (scores > 0).astype(int)

In [28]:
target_encoder.classes_[indices]

array([['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['face'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'],
       ['cat'